# Stochastic Differential Equations, Quantum Phase Space, and Julia 

$$
\def\julia{\texttt{julia}}
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\E{{\rm e}}
\def\dx{\D x}
\def\dt{\D t}
\def\dz{\D z}
\def\C{{\mathbb C}}
\def\R{{\mathbb R}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqqquad{\qquad\qquad}
\def\qqfor{\qquad\hbox{for}\qquad}
\def\qqwhere{\qquad\hbox{where}\qquad}
\def\Res_#1{\underset{#1}{\rm Res}}\,
\def\sech{{\rm sech}\,}
\def\vc#1{{\mathbf #1}}
$$

Dr. Ashton Bradley
<br>
ashton.bradley@otago.ac.nz
<br>
http://amoqt.otago.ac.nz

## Workshop 3 (1 hour): Quantum Phase space
Numerical solution of SDEs for physicists

- $\texttt{DifferentialEquations.jl}$: fast, adaptive stochastic integration
- Physical examples will include a selection of:
    - Damped, driven, harmonic oscillator
    - Phase diffusion
    - Model of a laser: stochastic Van der Pol equation
    - Intracavity parametric down conversion: squeezing in the +P representation


References

- _The Quantum World of Ultracold Atoms and Light: Book I: Foundations of Quantum Optics_, C. W. Gardiner and P. Zoller, [Imperial College Press, London (2014)](https://www.worldscientific.com/worldscibooks/10.1142/p941)
- _Generalized P-representations in quantum optics_, P. D. Drumond and C. W. Gardiner, [J. Phys. A-Math. and Gen., __13__, 2353-2368 (1980)](http://iopscience.iop.org/article/10.1088/0305-4470/13/7/018/meta)
- [A deep introduction to julia](http://ucidatascienceinitiative.github.io/IntroToJulia/), Christopher Rackauckas
- [_Stochastic Lifestyle_](http://www.stochasticlifestyle.com/), Christopher Rackauckas
- _Dynamics and statistical mechanics of ultra-cold Bose gases using c-field techniques_,
P. B. Blakie, A. S. Bradley, M. J. Davis, R. J. Ballagh, and C. W. Gardiner, [Advances in Phyiscs 57, 363 (2008)](https://doi.org/10.1080/00018730802564254)

# Master Equation to SDE

We now wish to utilize the link between FPE and SDE to map the quantum dynamical problem described by the density matrix equation of motion 

$$\frac{\partial \rho}{\partial t} = -\frac{i}{\hbar}[H,\rho] + {\cal L}(\rho)\tag{open quantum system}$$

to the numerically easier problem of solving a set of coupled SDE's. 

For our purpose, the essential feature is to understand how the action of a creation or annihilation operator on the density matrix maps to an equivalent term in the FPE. 

From there, provided the diffusion matrix is ___positive semi-definite___ ($\textrm{det}D\geq 1$), there is a mapping of the quantum problem to an equivalent SDE.

# P-function mappings
The simplest mapping is for the $P$-function, defined by the diagonal expansion in coherent states

$$\rho = \int d^2\alpha\; |\alpha\rangle\langle\alpha|P(\alpha,\alpha^*,t)$$

Provided such an expansion exists, we then have

$$a\rho = \int d^2\alpha\; |\alpha\rangle\langle\alpha|\alpha P(\alpha,\alpha^*,t)$$

and 

\begin{align}
a^\dagger\rho &= \int d^2\alpha\; P(\alpha,\alpha^*,t)\left(\alpha^*+\frac{\partial}{\partial \alpha}\right)|\alpha\rangle\langle\alpha|\notag\\
&=\int d^2\alpha\; |\alpha\rangle\langle\alpha|\left(\alpha^*-\frac{\partial}{\partial \alpha}\right)P(\alpha,\alpha^*,t)\notag
\end{align}

Notice here we have used integration by parts; we thus assume that $P$ decays sufficiently quickly as $|\alpha|\to\infty$ to justify this step. 

Carrying out similar operations for $\rho a$, $\rho a^\dagger$, we establish the first step in our mapping to SDE's

<div class="alert alert-block alert-warning">
<font color=blue>
\begin{align}
a\rho &\longrightarrow \alpha P\notag\\
a^\dagger\rho &\longrightarrow \left(\alpha^*-\frac{\partial}{\partial \alpha}\right)P\tag{P-function mappings}\\
\rho a&\longrightarrow \left(\alpha-\frac{\partial}{\partial \alpha^*}\right)P\notag\\
\rho a^\dagger &\longrightarrow \alpha^* P\notag
\end{align}
    </font></div>

In [1]:
using DifferentialEquations

# Damped harmonic oscillator

# Phase diffusion

# Intracavity parameteric down conversion